# 웹 페이지 추출
- 웹 페이지 추출 시 HTTP 헤더와 HTML의 meta 태그를 기반으로 인코딩 방식을 판별 필요
- 표준 라이브러리 urllib.request 모듈을 사용하여 웹 페이지 추출
- urllib.request에 포함돼 있는 urlopen() 함수에 url을 인자로 주면 웹 페이지 추출 가능
- HTTP 헤더를 변경 불가, Basic 인증을 사용하기 위한 복잡한 처리 필요
- HTTP 헤더 변경 및 Basic 인증을 위해 urllib 대신 Requests 모듈 사용 필요
---

## urllib을 이용한 웹 페이지 추출

In [1]:
from urllib.request import urlopen

In [75]:
f = urlopen('http://hanbit.co.kr')

In [33]:
# urlopen() 함수는 HTTPResponse 자료형의 객체를 반환
# 파일 객체이므로 open() 함수로 반환되는 파일 객체처럼 핸들링
type(f)

http.client.HTTPResponse

In [37]:
f.status   # 상태 코드를 추출   # 200 -> 정상 연결

200

In [38]:
f.getheader('Content-Type')    # HTTP 헤더 값 추출

'text/html; charset=UTF-8'

## HTTP 헤더에서 인코딩 방식 추출

In [79]:
import sys
from urllib.request import urlopen

In [80]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [81]:
# HTTP 헤더를 기반으로 인코딩 방식을 추출 ( 명시돼 있지 않을 경우 utf-8을 사용 )
encoding = f.info().get_content_charset(failobj='utf-8')

In [41]:
import sys
print(f'encoding : {encoding}', file=sys.stderr)

encoding : utf-8


In [76]:
text = f.read().decode(encoding)

In [77]:
print(text)

<!DOCTYPE html>
<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>한빛출판네트워크</title>
<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> 
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta property="og:type" content="website"/>
<meta property="og:title" content="한빛출판네트워크"/>
<meta property="og:description" content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀"/>
<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />
<meta property="og:url" content="https://hanbit.co.kr/"/>
<link rel="canonical" href="https://hanbit.co.kr/" />
<meta name="keywords" content="한빛미디어,한빛아카데미,한빛비즈,한빛라이프,한빛에듀,리얼타임,대관서비스,책,출판사,IT전문서,IT활용서,대학교재,경제경영,어린이/유아,실용/여행,전자책,자격증,교육,세미나,강의,ebook,정보교과서" />
<meta name="description" content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀" />
<me

## meta 태그에서 인코딩 방식 추출
- 웹 서버 설정에 따라 HTTP 헤더의 Content-Type 인코딩과 실제 사용되고 있는 인코딩 형식 상이
- 브라우저는 HTML 내부의 meta 태그 또는 응답 본문의 바이트열 확인을 통해 최종 인코딩 방식 결정 및 화면 출력
- 디코딩 처리에서 UnicodeDecodeError 발생 시 이러한 방식으로 구현 가능
- HTML meta에 명시되는 인코딩 형식

In [82]:
import re
import sys
from urllib.request import urlopen

In [83]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')
# bytes 자료형의 응답 본문을 일단 변수에 저장
bytes_content = f.read()

In [84]:
# charset은 HTML의 앞부분에 적혀 있는 경우가 많으므로
# 응답 본문의 앞부분 1024 바이트를 ASCII 문자로 디코딩
# ASCII 범위 이외의 문자는 U+FFFD ( REPLACEMENT CHARACTER ) 로 변환되어 예외가 발생하지 않음
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

In [87]:
# 디코딩한 문자열에서 정규 표현식으로 charset 값을 추출
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)

In [88]:
match.group(1)

'utf-8'

In [89]:
if match:
    encoding = match.group(1)
else:
    # charset이 명시되어 있지 않으면 UTF-8 사용
    encoding = 'utf-8'

## 정규표현식으로 스크레이핑 하기

In [91]:
import re

In [92]:
# re.search() 함수를 사용하면 두 번째 매개변수의 문자열이 
# 첫 번째 매개변수의 정규표현식과 매칭되는지 확인 가능
# 매칭되는 경우 Match 객체를 반환, 매칭되지 않는 경우 None 반환
re.search(r'a.*c', 'abc123DEF')


<re.Match object; span=(0, 3), match='abc'>

In [93]:
# 다음 예는 정규 표현식에 맞지 않으므로 None을 반환
re.search(r'a.*d', 'abc123DEF')

In [94]:
# re.ignorecase(또는 re.I)를 지정하면 대소문자를 무시
re.search(r'a.*d', 'abd123DEF', re.I)

<re.Match object; span=(0, 7), match='abd123D'>

In [95]:
# match 객체의 group() 메서드로 일치한 값을 추출
# 매개변수에 0을 지정하면 매치된 모든 값을 반환
m = re.search(r'a(.*)c', 'abc123DEF')
m.group(0)

'abc'

In [96]:
# 매개변수에 1 이상의 숫자를 지정하면 정규 표현식에서 0으로 감싼 부분에 해당하는 값을 추출
# 1이라면 1번째 그룹, 2라면 2번째 그룹 추출
m.group(1)

'b'

In [98]:
# re.findall() 함수를 사용하면 정규 표현식에 맞는 모든 문자열을 리스트 타입으로 추출 가능
# 다음 예는 2글자 이상의 단어를 모두 추출
# \w는 유니코드로 글자를 비교, 공백문자는 \s 등으로 추출 가능
re.findall(r'\w{2,}', 'This is a pen')

['This', 'is', 'pen']

In [99]:
# re.sub() 함수를 사용하면 정규 표현식에 매칭되는 문자열 치환
# 3번째 매개변수에 넣은 문자열에서 첫 번째 정규 표현식에 매칭되는 문자열을
# 2번째 매개변수 문자열로 치환함
re.sub(r'\w{2,}', 'That', 'This is a pen')

'That That a That'

In [102]:
result = re.search(r'a.*D', 'abc123DEF')
result

<re.Match object; span=(0, 7), match='abc123D'>

In [103]:
# match는 시작부터 일치하는지를 검사, search는 매칭되는 위치가 어디인지 탐색
result = re.match(r'a.*D', 'abc123DEF')
result

<re.Match object; span=(0, 7), match='abc123D'>